# Finding and plotting NetCDF data in the CEDA Archive

This Notebook demonstrates a few capabilities of the JASMIN Notebook service:
* Reading data from the CEDA Archive
* Analysing/processing data in the Notebook
* Plotting the results interactively
* Writing the outputs to your JASMIN `$HOME` directory
* Adding inline documentation

## Task 1: Find and read ECMWF ERA5 2-metre temperature data

This task involves two parts: (1) Finding the relevant ECMWF ERA5 file paths and (2) Reading a file path pattern into an `xarray Dataset` object.

Part (1) can be done in various ways. For the sake of simplicity, we have already searched the CEDA catalogue and found this dataset record:

https://catalogue.ceda.ac.uk/uuid/8aa70a91378d455ea63a2a1953858a7f

Following the "Download" link on the page reveals a browseable data path that reveals the 2-metre temperature data can be found under:

https://data.ceda.ac.uk/badc/ecmwf-era51/data/oper/an_sfc/2005/01/01 (e.g. file: `ecmwf-era51_oper_an_sfc_200501010000.2t.nc`)

On the file system, this translates to this pattern:

`/badc/ecmwf-era51/data/oper/an_sfc/2005/01/01/ecmwf-era51_oper_an_sfc_20050101*.2t.nc`
    
For Part (2)...

Import the `xarray` library as `xr`:

In [ ]:
import xarray as xr

Use `xr.open_mfdataset(...)` to read in the file pattern defined above to read in the hourly 2-metre temperature files.

In [ ]:
file_pattern = "/badc/ecmwf-era51/data/oper/an_sfc/2005/01/01/ecmwf-era51_oper_an_sfc_20050101*.2t.nc"

# Or you could make a generic template in a function
def get_template(year, month, day, variable):
    return (f"/badc/ecmwf-era51/data/oper/an_sfc/{year}/{month}/{day}/"
            f"ecmwf-era51_oper_an_sfc_{year}{month}{day}*.{variable}.nc")

year, month, day = "2005", "01", "01"
var_id = "2t" # 2-metre temperature
file_pattern = get_template(year, month, day, var_id)
file_pattern

Check we have some valid files using `glob`

In [ ]:
import glob
files = glob.glob(file_pattern)
len(files)

Open the files as a single `xarray Dataset` and take a look at its structure, attributes and time values:

In [ ]:
ds = xr.open_mfdataset(file_pattern)
ds

In [ ]:
ds.t2m.attrs

In [ ]:
ds.t2m.time

## Task 2: Calculate the max and min over all times in the dataset

You can access the 2-metre temperature variable using: `ds.t2m`.

The `ds.t2m Dataset` has its own methods: `max()` and `min()`. They require the argument `axis=0` in order to specify calculating the _max_ and _min_ only over the time axies.

Calculate two new variables:
1. `daily_max` - the maximum over all times
2. `daily_min` - the minimum over all times

In [ ]:
daily_max = ds.t2m.max(axis=0)
daily_min = ds.t2m.min(axis=0)

Check the `shape` attribute of the `daily_max` and `daily_min` variables to ensure they are 2D (i.e. `time` has been removed).

In [ ]:
daily_max.shape, daily_min.shape

## Task 3: Plot the daily maximum and minimum

The `daily_max` and `daily_min` variables now both have a `plot()` method. You can plot each of them individually.

In [ ]:
daily_max.plot()

In [ ]:
daily_min.plot()

You can difference them and plot a map of that variable

In [ ]:
diff = daily_max - daily_min
diff.plot()

## Task 4: Write the outputs to your JASMIN `$HOME` directory

Create an `outputs` directory in your `$HOME` directory

In [ ]:
import os
output_dir = f"{os.environ['HOME']}/outputs"

if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

Write the `daily_max` and `daily_min` variables to NetCDF files, using the `to_netcdf()` method on each variable. The method requires the output file path as the argument.


In [ ]:
daily_max.to_netcdf(f"{output_dir}/max_t2m.nc")

In [ ]:
daily_min.to_netcdf(f"{output_dir}/min_t2m.nc")

Check that the output files have been written to the `outputs` directory

In [ ]:
os.listdir(output_dir)

## Task 5: Adding inline documentation

For each cell, you can select either "Code", "Markdown" or "Raw" in the toolbar at the top of the notebook. If you select "Markdown", then the cell is no longer interpreted as Python code. Instead, it is interpreted as a mark-up language called `markdown`. This allows sophisticated formatting of text, images, code blocks etc., as described here:

https://guides.github.com/features/mastering-markdown/

In order to add Markdown cells to an existing notebook, click on the cell above the one you wish to annotate, then click the `+` button on the toolbar. Then change the cell format to "Markdown" in the drop-down menu.

Once you have completed the content, press `Shift+Enter` to see the formatted version. If you need to edit a formatted cell, just double-click into it.